### 1.1 TF-IDF文档表示
#### 一. 何为TF-IDF:  
1. TF-IDF-词频逆文档频率 (Term Frequency inverse Document Frequency)
2. TF-IDF把文章表示成一个向量.该向量的每个元素表示对应文章中该单词的出现次数(词频).如果单词表有10,000个单词, 则向量有10,000个元素,每个元素=词频*权重
3. TF-IDF是突出重要单词的一种文档表示方法. 因为像"the of"这样的单词, 在文章中会大量出现.如果直接用其词频放在向量中, 则这样的单词会对最终计算k临近距离产生较大影响.因此向量中每个元素使用词频*权重表示.何为重要单词, 需要满足以下两个条件:   
  1. 在本文章中大量出现(词频)  
  2. 在语料库中的其他文章很少出现(词频) 
  
#### 二. 强调重点单词 emphasize important words
1. 局部地方频繁出现 - appear frequently in document (common locally)   
 TF : Term Frequency词频 = {x,x,x ... x}(10,000个单词,每个单词出现的次数)
2. 语料库中很少出现 - rarely apperance in corpus (rarely global)  
 IDF : Inverse doc frequence逆文档频率 = $\log { \frac { \#docs }{ 1+\#doc\quad using\quad word }  } = \log { \frac { 语料库总文档数 }{ 1+使用该单词的文档数 }  }  $
3. 文档表示向量 = $TF*IDF$ (词频*权重)

#### 三. TF-IDF关键词提取
 1. 使用逆文档词频表示文档后, 计算每个单词在corpus中权值的均值.大于这个均值的就是该文档的关键词

### 1.2 TextRank

#### 一. graph模型
1. TextRank是基于graph的ranking模型.这类模型的最根本任务是决定graph中顶点的重要性  
2. 有向无权图计算顶点重要性的方法  
 图$G=(V,E)$是顶点和边构成的有向图, 顶点$V_i$的得分为$$S\left( { V }_{ i } \right) =\left( 1-d \right) +d*\sum _{ j\in In\left( { V }_{ i } \right)  }^{  }{ \frac { S\left( { V }_{ j } \right)  }{ \left| Out\left( { V }_{ j } \right)  \right|  }  } $$  
  1. $In\left( { V }_{ i } \right) $ : $V_i$的入边集合, 所以$V_j$是所有指向$V_i$的点  
  2. $\left| Out\left( { V }_{ j } \right)  \right| $ : 顶点${ V }_{ j }$出边的个数.
  2. $d$ : 阻尼系数, 一般设置为0.85, 指用户有多大的概率从当前页面跳转到与之关联的其他链接上  
  
3. 无向带权图计算顶点的分的方法  
  顶点$V_i$的得分计算公式 : $$WS\left( { V }_{ i } \right) =\left( 1-d \right) +d*\sum _{ { V }_{ i }\in In\left( { V }_{ i } \right)  }^{  }{ \left[ \frac { { W }_{ ji } }{ \sum _{ { V }_{ k }\in Out\left( { V }_{ j } \right)  }^{  }{ { W }_{ jk } }  } WS\left( { V }_{ j } \right)  \right]  } $$  
   1. ${ W }_{ ji }$ : 顶点$V_j$到$V_i$的边的权值
   2. $Out\left( { V }_{ j }\right)$ : 顶点$V_j$的出边集合
2. 如何计算每个顶点的得分  
 1. 从公式中可以看出, 顶点的得分依赖于指向该点的顶点的得分. 是一个循环依赖问题.  
 2. 但是可以随机初始化每个顶点的得分, 再迭代使用上述公式计算每个顶点的得分, 直到其收敛  
 3. 迭代一定会收敛的原因 : 上述公式的$\left| Out\left( { V }_{ j } \right)  \right| $可看做马尔科夫转移概率, 所以多次计算后会收敛到稳定状态  
 
#### 二. 关键词抽取
1. 关键词抽取的作用  
 关键词抽取, 是从文章中提取出来的一系列关键词.这些关键词形成一个实体用来对文章建立索引, 对文章分类, 或者是对文章简明摘要
2. 共生关系的定义  
 我们使用2个单词同时出现的词距$N$表示他们的"共生关系", 词距$N$一般设置为$(2,10)$. 即只要2个词距在N之内, 就说明2个词有共生关系  
3. TextRank如何对文本构建graph  
  1. 将文本分词后形成一些列tocken, 经过句法过滤器过滤掉一部分tocken. (过滤器可以是只选择名词+动词,或只选则名词+形容词). 这些过滤后的tocken做为graph中的顶点  
  2. 边 : 只要词存在共生关系, 就在graph中增加一条边. ( 最终形成的graph是无向不带权重的图 )  
4. TextRank如何计算顶点的分  
  首先, 初始化所有顶点得分为1. 然后迭代计算直到收敛. 最后选择top-T的顶点作为关键词 ($T$一般为5到20或设置为$\frac { 文本长度 }{ 3 } $)
5. 如上操作, 我们并没有对文本进行标注, 只是分词后构建好graph迭代计算即可. 因此, TextRank是无监督算法
6. 中文TextRank可使用`jieba分词`

#### 三. 关键句抽取
1. 关键句抽取, graph的顶点为文本中的每句话
2. 句子间的共生关系 :  
 和词的共生关系不同, 句子间隔对句子的表达没有什么关系. 因此, 转而使用"两个句子表达的意思是否有交叉来判定共生关系". 如果意思有交叉, 则2个句子有共生关系. 句子意思有交叉又叫句子间的相似度, 即句子$S_i$和$S_j$的相似度为$$Similarity\left( { S }_{ i },{ S }_{ j } \right) =\frac { \left| { W }_{ k }|{ W }_{ k }\in { S }_{ i }\& { W }_{ k }\in { S }_{ j } \right|  }{ \log { \left| { S }_{ i } \right|  } +\log { \left| { S }_{ j } \right|  }  } $$  
   1. 分子 : 即属于S_i又属于S_j的单词个数  
   2. 分母 : 为避免句子太长造成重叠单词个数很多而出现的正则化项  
3. 关键句graph构建  
  顶点为文章中的句子, 边为两个句子的相似度, 因此形成的graph为无向带权图. 计算方式采用无向带权图的顶点得分计算公式迭代
  
#### 四. jieba提取关键字

In [71]:
import jieba
import jieba.analyse
sentence = '中新网北京12月1日电(记者 张曦) 30日晚，高圆圆和赵又廷在京举行答谢宴，诸多明星现身捧场，其中包括张杰(微博)、谢娜(微博)夫妇、何炅(微博)、蔡康永(微博)、徐克、张凯丽、黄轩(微博)等高圆圆身穿粉色外套，看到大批记者在场露出娇羞神色，赵又廷则戴着鸭舌帽，十分淡定，两人快步走进电梯，未接受媒体采访.记者了解到，出席高圆圆、赵又廷答谢宴的宾客近百人，其中不少都是女方的高中同学'
print("sentence : ",sentence)
jieba.analyse.textrank(sentence, topK=10, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v'))
# print(','.join(jieba.cut(sentence,HMM=False)))

sentence :  中新网北京12月1日电(记者 张曦) 30日晚，高圆圆和赵又廷在京举行答谢宴，诸多明星现身捧场，其中包括张杰(微博)、谢娜(微博)夫妇、何炅(微博)、蔡康永(微博)、徐克、张凯丽、黄轩(微博)等高圆圆身穿粉色外套，看到大批记者在场露出娇羞神色，赵又廷则戴着鸭舌帽，十分淡定，两人快步走进电梯，未接受媒体采访.记者了解到，出席高圆圆、赵又廷答谢宴的宾客近百人，其中不少都是女方的高中同学


['记者', '看到', '接受', '媒体', '了解', '现身', '捧场', '包括', '采访', '外套']

In [51]:
# 将GBK的停用词表转换成utf8
content =  open("/Users/lj/IdeaProjects/DeepLearning/blog/自然语言处理/data/stop_words.txt",'rb').read()
new_content = content.decode('GBK').encode('utf-8')
open("/Users/lj/IdeaProjects/DeepLearning/blog/自然语言处理/data/stop_words_utf8.txt",'wb').write(new_content)

8115